In [ ]:
!pip install rarfile
%tensorflow_version 2.x
import tensorflow
import cv2
import io
import numpy as np
import pandas as pd
import rarfile
import numpy as np
from google.colab import drive
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
from google.colab.patches import cv2_imshow
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
tensorflow.__version__

drive.mount('/content/gdrive')

#put material path here
path = "/content/gdrive/MyDrive/Emotions_detection/Material.rar"

zip_object = rarfile.RarFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close

VIDEO_HTML = """
<video autoplay
 width=%d height=%d style='cursor: pointer;'></video>
<script>

var video = document.querySelector('video')

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)
  
var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""

def take_picture(filename="capture_test.jpg", quality=2, size=(400,300)):

    display(HTML(VIDEO_HTML % (size[0],size[1],quality)))
    data = eval_js("data")
    binary = b64decode(data.split(",")[1])
    f = io.BytesIO(binary)
    return np.asarray(Image.open(f))

img_take = take_picture()
img_take = cv2.cvtColor(img_take, cv2.COLOR_BGR2RGB)
cascade_faces = "/content/Material/haarcascade_frontalface_default.xml"
model_path = "/content/Material/modelo_01_expressoes.h5"
face_detection = cv2.CascadeClassifier(cascade_faces)
emotional_classifier = load_model(model_path, compile=False)
expre = ["Anger", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
original = img_take.copy()
faces = face_detection.detectMultiScale(original,scaleFactor=1.1,minNeighbors=3,minSize=(20,20))
grey = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

if len(faces) > 0:
    for fx, fy, fw, fh in faces:
        roi = grey[fy:fy + fh, fx:fx + fw]
        roi = cv2.resize(roi, (48,48))
        roi = roi.astype("float")/255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        pred = emotional_classifier.predict(roi)[0]
        print(pred)
        emotion_prob = np.max(pred)
        label = expre[pred.argmax()]
        cv2.putText(original, label, (fx, fy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.rectangle(original, (fx, fy), (fx + fw, fy + fh),(0, 0, 255), 2)
else:
    print("No detected face")

cv2_imshow(original)

probabilities = np.ones((250, 300, 3), dtype="uint8")*255

if len(faces) == 1:
    for i, (emotion, proba) in enumerate(zip(expre, pred)):
        txt = "{}: {:2f}%".format(emotion, proba*100)
        w = int(proba*300)
        cv2.rectangle(probabilities, (7, (i,*35) + 5), (w, (i*35)+35), (200,250,20), -1)
        cv2.putText(probabilities, txt, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 0), 1, cv2.LINE_AA)
    cv2_imshow(probabilities)
cv2.imwrite("Capture.jpg", original)
cv2.destroyAllWindows()

# Nova seção